In [1]:
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'AffectNet_train_set/seven/'
validation = ALL_DATA_DIR + 'AffectNet_val_set/seven/'

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
training_s = tf.keras.utils.image_dataset_from_directory(training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)
validation_s = tf.keras.utils.image_dataset_from_directory(validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 283901 files belonging to 7 classes.
Found 3500 files belonging to 7 classes.


In [5]:
base_model = keras.applications.MobileNetV3Small(input_shape=(224,224,3), include_top = False, weights="imagenet")

In [6]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [7]:
for layer in base_model.layers:
    layer.trainable = False

In [8]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(training_s, validation_data=validation_s, epochs=5)

Epoch 1/5
8872/8872 [==============================] - 245s 27ms/step - loss: 1.1353 - accuracy: 0.5847 - val_loss: 2.2067 - val_accuracy: 0.2591
Epoch 2/5
8872/8872 [==============================] - 224s 25ms/step - loss: 1.0769 - accuracy: 0.6074 - val_loss: 2.1123 - val_accuracy: 0.2803
Epoch 3/5
8872/8872 [==============================] - 226s 25ms/step - loss: 1.0706 - accuracy: 0.6092 - val_loss: 2.1458 - val_accuracy: 0.2769
Epoch 4/5
8872/8872 [==============================] - 225s 25ms/step - loss: 1.0678 - accuracy: 0.6106 - val_loss: 2.1829 - val_accuracy: 0.2826
Epoch 5/5
8872/8872 [==============================] - 225s 25ms/step - loss: 1.0670 - accuracy: 0.6109 - val_loss: 2.1787 - val_accuracy: 0.2811


In [10]:
for layer in base_model.layers:
    layer.trainable = True

In [11]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

In [12]:
es = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=30, callbacks=[es])

Epoch 1/30
8872/8872 [==============================] - 549s 61ms/step - loss: 0.7657 - accuracy: 0.7251 - val_loss: 1.5353 - val_accuracy: 0.4740
Epoch 2/30
8872/8872 [==============================] - 545s 61ms/step - loss: 0.6402 - accuracy: 0.7667 - val_loss: 1.3796 - val_accuracy: 0.5283
Epoch 3/30
8872/8872 [==============================] - 545s 61ms/step - loss: 0.5975 - accuracy: 0.7811 - val_loss: 1.4105 - val_accuracy: 0.5257
Epoch 4/30
8872/8872 [==============================] - 544s 61ms/step - loss: 0.5642 - accuracy: 0.7925 - val_loss: 1.3496 - val_accuracy: 0.5497
Epoch 5/30
8872/8872 [==============================] - 538s 61ms/step - loss: 0.5333 - accuracy: 0.8034 - val_loss: 1.4060 - val_accuracy: 0.5457
Epoch 6/30
8872/8872 [==============================] - 545s 61ms/step - loss: 0.5022 - accuracy: 0.8143 - val_loss: 1.4863 - val_accuracy: 0.5446
Epoch 7/30
8872/8872 [==============================] - 543s 61ms/step - loss: 0.4727 - accuracy: 0.8248 - val_loss: 1

In [14]:
model.evaluate(training_s)

8872/8872 [==============================] - 224s 25ms/step - loss: 0.5413 - accuracy: 0.8009


[0.5413046479225159, 0.8008989095687866]

In [15]:
model.evaluate(validation_s)

110/110 [==============================] - 3s 23ms/step - loss: 1.3496 - accuracy: 0.5497


[1.3496047258377075, 0.5497142672538757]

In [16]:
model.save("0926_mobile_affect_eval.h5")